In [1]:
from keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from echotrain.utils.handling_yaml import load_config_file
from echotrain.training.trainer_base import TrainerBase

Load Dataset

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train[:5000] / 255.0
y_train = to_categorical(y_train[:5000])

x_test = x_test[:1000] / 255.0
y_test = to_categorical(y_test[:1000])

print("x_train :", x_train.shape)
print("y_train :", y_train.shape)
print("x_test :", x_test.shape)
print("y_test :", y_test.shape)

x_train : (5000, 32, 32, 3)
y_train : (5000, 10)
x_test : (1000, 32, 32, 3)
y_test : (1000, 10)


Create Train And Validation Generators

In [3]:
def train_generator(batch_size):
    datagen_train = ImageDataGenerator(
        width_shift_range=3 / 32,
        height_shift_range=3 / 32,
        brightness_range=(0.3, 1.8),
        horizontal_flip=True,
    )
    train_gen = datagen_train.flow(x_train, y_train, batch_size=batch_size)
    return train_gen


def val_generatior(batch_size):
    datagen_validation = ImageDataGenerator(
        width_shift_range=3 / 32,
        height_shift_range=3 / 32,
        brightness_range=(0.3, 1.8),
        horizontal_flip=True,
    )

    val_gen = datagen_validation.flow(x_test, y_test, batch_size=batch_size)
    return val_gen

Define Sample Model

In [4]:
def create_model():
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPool2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', ))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', ))
    model.add(layers.MaxPool2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', ))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', ))
    model.add(layers.MaxPool2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

Test The trainer_base

In [6]:
model = create_model()

config = load_config_file('../config/config_example.yaml')
batch_size = config.trainer.epochs
train_gen = train_generator(batch_size)
val_gen = val_generatior(batch_size)
train_steps = int(x_train.shape[0] / batch_size)
val_steps = int(x_test.shape[0] / batch_size)
model_name = 'unet'
base_addr = f'Experience/{model_name}'
trainer = TrainerBase(base_addr, config.trainer)
trainer.train(model, train_data_gen=train_gen, val_data_gen=val_gen,
                n_iter_train=train_steps, n_iter_val=val_steps)

/model.{epoch:03d}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5
Epoch 4/100
50/50 [==============================] - 28s 535ms/step - loss: 1.7820 - accuracy: 0.3736 - val_loss: 1.6921 - val_accuracy: 0.4020

KeyboardInterrupt: 

Test export method

In [7]:
trainer.export()

Experience/unet/checkpoints/model.005-1.62-0.41.hdf5
